In [1]:
#!/usr/bin/env python
# coding: utf-8
from data_chooser import interactive_data_chooser, create_fake_df


from dash import Dash, dcc, html, Input, Output
import plotly.express as px


df = create_fake_df(33)

chooser = interactive_data_chooser(df, df.columns)
chooser.activate_plot()


     x   y1  y2  manual_outlier  model_outlier
0   98    3   9              -1              0
1   59   27  48              -1              0
2   76    0   5              -1              0
3   22   59  30              -1              0
4   27   30   3              -1              0
5   83   79  22              -1              0
6   16   85  55              -1              0
7   15   57  20              -1              0
8   29   98  39               1              0
9   29   27  67              -1              0
10  69   69  33              -1              0
11  89   52  40              -1              0
12  52   66  93              -1              0
13  68   62  76              -1              0
14  30   91  24               1              0
15  92   62  17              -1              0
16  86   79  41              -1              0
17  85   90  58              -1              0
18   7   76   3              -1              0
19  62   90  61              -1              0
20  15   39  

In [22]:
chooser.outlier_df

,x,y1,y2
